In [105]:
import os
import re
import json
import requests
from datetime import timedelta, date, datetime

import folium
from pyproj import Transformer
from folium import plugins
from shapely import geometry
from shapely.geometry import Point, LineString, Polygon, MultiPolygon
from random import uniform
from sklearn.neighbors import DistanceMetric

import osmapi as osm
from openrouteservice import client

# Анализ пространственных данных. Домашнее задание №2

Мягкий дедлайн: __4 ноября 2020 г. 23:59__

Жесткий дедлайн (со штрафом в _50%_ от количества набранных вами за ДЗ баллов): __5 ноября 2020 г. 08:59__

Визуализация "чего-либо" __без__ выполненного основного задания оценивается в __0 баллов__

ФИО: Тарелкин Александр Алексеевич

Группа: MADE-ML-11

## Задание №1. Горячая точка (алгоритм - 10 баллов, визуализация - 10 баллов).

Генерируйте рандомные точки на планете Земля до тех пор, пока не попадете на территорию ``Афганистана``

1. Вы можете использовать функции принадлжености точки полигону и расстояния от точки до полигона (в метрах)
2. Предложите не наивный алгоритм поиска (генерировать __напрямую__ точку из полигона границ Афганистана __запрещено__)

#### Полигон Афганистана

In [6]:
AF_RELATION_ID = 303427

api = osm.OsmApi()

relation = api.RelationGet(AF_RELATION_ID)

ways_ids = [x['ref'] for x in relation['member'] if x['type'] == 'way'] # получаем id путей границы
ways = api.WaysGet(ways_ids) # получаем сами пути

nodes_ids = [y for x in ways.values() for y in x['nd']] # получаем id точек границы
nodes_ids = [x for i,x in enumerate(nodes_ids) if i % 5 == 0] # берем каждую пятую точку границы, для быстродействия 

In [133]:
coords = {}
for i, x in enumerate(nodes_ids):
    if i % 100 == 0:
        print(i, f' / {len(nodes_ids)}')
    node = api.NodeGet(x)
    coords[i] = (node['lat'], node['lon'])
    
c = [x for x in coords.values()]

af_poly = Polygon(c).convex_hull # полигон Афганистана

0  / 9292
100  / 9292
200  / 9292
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
300  / 9292
400  / 9292
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
500  / 9292
600  / 9292
700  / 9292
800  / 9292
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
900  / 9292
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
1000  / 9292
1100  / 9292
1200  / 9292
1300  / 9292
1400  / 9292
1500  / 9292
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
1600  / 9292
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
1700  / 9292
1800  / 9292
('Connection aborted.

#### Генерация случайных точек. Наивный метод.

In [141]:
# наивный метод

up_boundary_lat = 90
down_boundary_lat = -90
up_boundary_lon = 180
down_boundary_lon = -180

m = folium.Map(
    location=[af_poly.centroid.x, af_poly.centroid.y],
    zoom_start=1,
)

p = Point((uniform(down_boundary_lat, up_boundary_lat), uniform(down_boundary_lon, up_boundary_lon)))  
i = 0
while not af_poly.contains(p):
    p = Point((uniform(down_boundary_lat, up_boundary_lat), uniform(down_boundary_lon, up_boundary_lon)))
    i += 1
    folium.Marker(
        location=[p.x, p.y],
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m)
    
print(i, 'iteration')

1010 iteration


In [142]:
m.save(os.path.join('Plugins_1.html'))
m # визуализация - карта, с отметками сгенерированных точек

#### Генерация точек. Не наивный метод.

In [135]:
def calc_haversine_dist_from_point_to_Af(lat, lon):
    """ возвращает расстояние от точки до центра Афганистана в км """
    
    af_center_coords = list(map(lambda x: x*3.1415925/180, [af_poly.centroid.x, af_poly.centroid.y]))
    point_coords = list(map(lambda x: x*3.1415925/180, [lat, lon]))
    
    haversine = DistanceMetric.get_metric('haversine')
    
    return 6371 * haversine.pairwise([af_center_coords, point_coords])[0][1]

In [157]:
# не наивный алгоритм - иттеративно уменьшаем область, опираясь на расстояние от точки до центра полигона

x_min, x_max, y_min, y_max = down_boundary_lat, up_boundary_lat, down_boundary_lon, up_boundary_lon

p = Point((uniform(x_min, x_max), uniform(y_min, y_max))) 
coords = [[p.x, p.y]]
i = 1

while not af_poly.contains(p):
    i += 1
    corr = calc_haversine_dist_from_point_to_Af(p.x, p.y) * 180 / (6371 * 3.14159265) # расстояние в градусах до центра полигона
    
    # рассчитываем новые гранцы области
    
    x_min = max(p.x - 1.5 * corr, x_min)
    x_max = min(p.x + 1.5 * corr, x_max)
    y_min = max(p.y - 1.5 * corr, y_min)
    y_max = min(p.y + 1.5 * corr, y_max)
    
    print('x_min = ', x_min,' x_max = ', x_max, ' y_min = ', y_min, ' y_max = ', y_max)
    
    p = Point((uniform(x_min, x_max), uniform(y_min, y_max)))
    
    coords.append([p.y, p.x]) # массив координат точек, для последующей визуализации
    
    if i > 20:
        x_min, x_max, y_min, y_max = down_boundary_lat, up_boundary_lat, down_boundary_lon, up_boundary_lon

print(i, 'iteration') 

x_min =  -90  x_max =  90  y_min =  -8.184121567044315  y_max =  180
x_min =  2.253032904720044  x_max =  83.80070769268741  y_min =  58.97513885177166  y_max =  140.522813639739
x_min =  2.253032904720044  x_max =  83.80070769268741  y_min =  60.06136669085336  y_max =  140.522813639739
x_min =  2.253032904720044  x_max =  51.3447392148572  y_min =  60.06136669085336  y_max =  116.92708555214475
x_min =  2.253032904720044  x_max =  44.1804187786055  y_min =  60.06136669085336  y_max =  98.01295172618569
x_min =  2.253032904720044  x_max =  44.1804187786055  y_min =  60.06136669085336  y_max =  98.01295172618569
x_min =  14.908451679464967  x_max =  38.635832091649064  y_min =  60.06136669085336  y_max =  75.11799507099077
x_min =  17.084139898408154  x_max =  38.27002063086239  y_min =  60.06136669085336  y_max =  75.11799507099077
x_min =  17.084139898408154  x_max =  38.27002063086239  y_min =  61.228824857381355  y_max =  75.11799507099077
x_min =  17.084139898408154  x_max =  38.2

In [158]:
# Анимированная визуализация

start_date = datetime.today()

times = [single_date.strftime('%Y-%m-%d') for single_date in (start_date + timedelta(n) for n in range(len(coords)))]

points = [
    {
        'time': time,
        'coordinates': point
    } for time, point in zip(times, coords)
]

features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': point['coordinates'],
        },
        'properties': {
            'time': point['time'],
            'id': 'point',
            'icon': 'marker',
            'iconstyle': {
                'iconUrl': 'https://leafletjs.com/examples/geojson/baseball-marker.png',
                'iconSize': [20, 20]
            }
        }
    } for point in points
]

features.append(
    {
        'type': 'Feature',
        'geometry': {
            'type': 'LineString',
            'coordinates': coords,
        },
        'properties': {
            'popup': 'Current address',
            'times': times,
            'icon': 'circle',
            'iconstyle': {
                'fillColor': 'green',
                'fillOpacity': 0.6,
                'stroke': 'false',
                'radius': 2
            },
            'style': {'weight': 0},
            'id': 'man'
        }
    }
)

m = folium.Map(
    location=[af_poly.centroid.x, af_poly.centroid.y],
    tiles='cartodbpositron',
    zoom_start=1,
)

plugins.TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': features
    },
    period='P1D',
    add_last_point=True,
    auto_play=False,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options='YYYY/MM/DD',
    time_slider_drag_update=True,
    duration='P13D'
).add_to(m)

m.save(os.path.join('Plugins_2.html'))
m

Визуализируйте пошагово предложенный алгоритм при помощи ``Folium``

## Задание №2. Качество жизни (20 баллов).

Для измерения показателя качества жизни в точке, найденной в предыдущем задании, вам необходимо рассчитать следующую сумму расстояний (в метрах):

1. Расстояние от точки до 5 ближайших __*__ банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости
2. Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице
3. Расстояние от точки до 5 ближайших кинотеатров, наодящихся в стране с самым большим отношением числа железнодорожных станций к автобусным остановкам в южной части __**__

__*__ При поиске _N_ ближайших объектов обязательно использовать ``R-tree``

__**__ Южной частью страны является территория, находящаяся к югу от множества точек, равноудаленных от самой северной и самой южной точек страны

In [ ]:
# put your code here

## Задание №3. Поездка по Нью-Йорку (маршрут - 20 баллов, визуализация - 10 баллов).

Добраться __на автомобиле__ от входа в ``Central Park`` __Нью-Йорка__ (со стороны ``5th Avenue``) до пересечения ``Water Street`` и ``Washington Street`` в Бруклине (откуда получаются лучшие фото Манхэттенского моста) довольно непросто - разумеется, из-за вечных пробок. Однако еще сложнее это сделать, проезжая мимо школ, где дети то и дело переходят дорогу в неположенном месте.

Вам необходимо построить описанный выше маршрут, избегая на своем пути школы. Визуализируйте данный маршрут (также добавив школы и недоступные для проезда участки дорог) при помощи ``Folium``

Данные о расположении школ Нью-Йорка можно найти [здесь](https://catalog.data.gov/dataset/2019-2020-school-point-locations)

In [130]:
url = 'https://data.cityofnewyork.us/api/views/a3nt-yts4/rows.json?accessType=DOWNLOAD'

def CreateBufferPolygon(point_in, resolution=10, radius=100):
    
    transformer = Transformer.from_proj(proj_from=4326, proj_to=32632, always_xy=True) # WGS84 UTM32N
    point_in_proj = transformer.itransform([point_in]) # unpack list to arguments
    point_buffer_proj = Point(point_in_proj).buffer(radius, resolution=resolution) # 100 m buffer
    
    # Iterate over all points in buffer and build polygon

    transformer_back = Transformer.from_proj(proj_from=32632, proj_to=4326, always_xy=True)

    poly_wgs = list(transformer_back.itransform(point_buffer_proj.exterior.coords))  # Transform back to WGS84

    return poly_wgs

def style_function(color):
    return lambda feature: dict(color=color,
                              weight=3,
                              opacity=0.75)

In [123]:
# make reasonable assumption about boundaries of region considered 

x_min = 40.6845
x_max = 40.8549
y_min = -74.0686
y_max = -73.9006

NY_school_json = requests.get(url).json() # Get data in json format

school_coords= [list(map(float, re.findall(r'-*\d+\.*\d*', coords[8]))) for coords in NY_school_json['data']] # Parse coords

# Leave schools only in region considered
school_coords_filtred = list(filter(lambda c: c[1] > x_min and c[1] < x_max and c[0] > y_min and c[0] < y_max, coords))

school_coords_filtred.reverse()

In [159]:
api_key = '5b3ce3597851110001cf62484be23207328a4f5fae21d9f5751234f5' # Individual api key
clnt = client.Client(key=api_key) # Create client with api key

map_params = {'tiles':'Stamen Toner',
              'location':([40.746477, -73.978157]),
              'zoom_start': 8}
map1 = folium.Map(**map_params)

# Populate a schools buffer polygon list
schools_poly = []
for school_data in school_coords_filtred:
    
    folium.features.Marker(school_data,
                           popup=f'school<br>{school_data}').add_to(map1)
    
    # Create buffer polygons around schools with 150 m radius and low resolution
    school_poly_coords = CreateBufferPolygon(school_data,
                                           resolution=2, # low resolution to keep polygons lean
                                           radius=150)
    schools_poly.append(school_poly_coords)
    
    school_poly_coords = [(y,x) for x,y in school_poly_coords] # Reverse coords for folium/Leaflet
    folium.vector_layers.Polygon(locations=school_poly_coords,
                                  color='#ffd699',
                                  fill_color='#ffd699',
                                  fill_opacity=0.2,
                                  weight=3).add_to(map1)
    
schools_poly = list(map(Polygon, schools_poly))

# Request normal route between appropriate locations avoiding schools
request_params = {'coordinates': [[-73.97215, 40.76545], 
                                 [-73.98957, 40.70319]],
                'format_out': 'geojson',
                'profile': 'driving-car',
                'preference': 'shortest',
                'instructions': 'false',
                 'options': {'avoid_polygons': geometry.mapping(MultiPolygon(schools_poly))}}

route_detour = clnt.directions(**request_params)

folium.features.GeoJson(data=route_detour,
                        name='Route avoiding schools',
                        style_function=style_function('#FF0000'),
                        overlay=True).add_to(map1)

map1.add_child(folium.map.LayerControl())
map1.save(os.path.join('Plugins_3.html'))
map1